In [17]:
from bs4 import BeautifulSoup
import requests
import time


In [18]:
base_url = 'https://www.fbi.gov'

main_url = f'{base_url}/wanted/cyber'
response = requests.get(main_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [19]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0 Safari/537.36'
}
response = requests.get(main_url, headers=headers)


In [20]:
cards = soup.find_all('div', class_='portal-type-person')

In [21]:
import csv
data=[]
    


In [22]:
for index, card in enumerate(cards, start=1):
    name = card.find('h3').text.strip()

    relative_link = card.find('a')['href']
    profile_url = f'{base_url}{relative_link}' if relative_link.startswith('/') else relative_link

    img_tag = card.find('img')
    img_url = f"{base_url}{img_tag['src']}" if img_tag and img_tag['src'].startswith('/') else img_tag['src'] if img_tag else ''

    profile_response = requests.get(profile_url)
    profile_soup = BeautifulSoup(profile_response.text, 'html.parser')

    description_div = profile_soup.find('div', {'class': 'wanted-person-description'})
    aliases = ""
    nationality = ""
    date_of_birth = ""
    place_of_birth = ""
    charges = ""

    details = profile_soup.find_all('div', class_='wanted-person-details')
    for detail_section in details:
        items = detail_section.find_all('li')
        for item in items:
            text = item.get_text(separator=":").strip()
            if 'Aliases' in text:
                aliases = text.split(":", 1)[-1].strip()
            elif 'Date(s) of Birth Used' in text:
                date_of_birth = text.split(":", 1)[-1].strip()
            elif 'Place of Birth' in text:
                place_of_birth = text.split(":", 1)[-1].strip()
            elif 'Nationality' in text:
                nationality = text.split(":", 1)[-1].strip()
            elif 'Charges' in text:
                charges = text.split(":", 1)[-1].strip()

    data.append([
        name,
        profile_url,
        img_url,
        aliases,
        date_of_birth,
        place_of_birth,
        nationality,
        charges
    ])

    print(f"[{index}/{len(cards)}] Scraped: {name}")

    time.sleep(1)


In [24]:
with open('cyber_wanted_full.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Name', 'Profile URL', 'Image URL', 'Aliases', 'DOB', 'Place of Birth', 'Nationality', 'Charges'])
    writer.writerows(data)

print(f"\n✅ Done! Saved {len(data)} entries to 'cyber_wanted_full.csv'")



✅ Done! Saved 0 entries to 'cyber_wanted_full.csv'


In [25]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Base URL
base_url = 'https://www.fbi.gov'
main_url = f'{base_url}/wanted/cyber'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

# Request main page
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Use updated li class
cards = soup.find_all('li', class_='portal-type-person')

print(f"Found {len(cards)} cards")

data = []

for i, card in enumerate(cards, start=1):
    # Get name
    name_tag = card.find('h3', class_='title')
    name = name_tag.text.strip() if name_tag else 'N/A'

    # Profile URL
    profile_link = name_tag.find('a')['href']
    profile_url = profile_link if profile_link.startswith('http') else f"{base_url}{profile_link}"

    # Image URL
    img_tag = card.find('img')
    img_url = img_tag['src'] if img_tag else 'N/A'

    # Visit profile page
    profile_resp = requests.get(profile_url, headers=headers)
    profile_soup = BeautifulSoup(profile_resp.text, 'html.parser')

    # Extract details
    aliases = dob = pob = nationality = charges = 'N/A'

    details = profile_soup.find_all('div', class_='wanted-person-details')
    for detail_block in details:
        items = detail_block.find_all('li')
        for item in items:
            text = item.get_text(separator=":").strip()
            if 'Aliases' in text:
                aliases = text.split(':', 1)[-1].strip()
            elif 'Date(s) of Birth Used' in text:
                dob = text.split(':', 1)[-1].strip()
            elif 'Place of Birth' in text:
                pob = text.split(':', 1)[-1].strip()
            elif 'Nationality' in text:
                nationality = text.split(':', 1)[-1].strip()
            elif 'Charges' in text:
                charges = text.split(':', 1)[-1].strip()

    data.append([
        name, profile_url, img_url, aliases, dob, pob, nationality, charges
    ])

    print(f"[{i}/{len(cards)}] Scraped: {name}")
    time.sleep(1)  # be polite to the server

# Write to CSV
with open('cyber_wanted_full.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Name', 'Profile URL', 'Image URL', 'Aliases', 'DOB', 'Place of Birth', 'Nationality', 'Charges'])
    writer.writerows(data)

print(f"\n✅ Done! Saved {len(data)} entries to 'cyber_wanted_full.csv'")


Found 0 cards

✅ Done! Saved 0 entries to 'cyber_wanted_full.csv'


In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv
import time

# Setup Selenium
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in background
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the FBI Cyber Wanted page
driver.get("https://www.fbi.gov/wanted/cyber")
time.sleep(5)  # Wait for JS to load

# Get rendered page source
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Find wanted person cards
cards = soup.find_all('li', class_='portal-type-person')
print(f"Found {len(cards)} cards.")

data = []

for card in cards:
    name_tag = card.find('h3', class_='title')
    name = name_tag.text.strip() if name_tag else 'N/A'
    profile_link = name_tag.find('a')['href']
    profile_url = profile_link if profile_link.startswith('http') else f"https://www.fbi.gov{profile_link}"
    img_tag = card.find('img')
    img_url = img_tag['src'] if img_tag else ''

    # Fetch full profile page
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(profile_url)
    time.sleep(3)
    profile_soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # Extract details
    aliases = dob = pob = nationality = charges = 'N/A'
    details = profile_soup.find_all('div', class_='wanted-person-details')
    for detail_block in details:
        items = detail_block.find_all('li')
        for item in items:
            text = item.get_text(separator=":").strip()
            if 'Aliases' in text:
                aliases = text.split(':', 1)[-1].strip()
            elif 'Date(s) of Birth Used' in text:
                dob = text.split(':', 1)[-1].strip()
            elif 'Place of Birth' in text:
                pob = text.split(':', 1)[-1].strip()
            elif 'Nationality' in text:
                nationality = text.split(':', 1)[-1].strip()
            elif 'Charges' in text:
                charges = text.split(':', 1)[-1].strip()

    data.append([name, profile_url, img_url, aliases, dob, pob, nationality, charges])
    print(f"Scraped: {name}")

# Save to CSV
with open('cyber_wanted_full.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Name', 'Profile URL', 'Image URL', 'Aliases', 'DOB', 'Place of Birth', 'Nationality', 'Charges'])
    writer.writerows(data)

print(f"\n✅ Done! Saved {len(data)} entries to 'cyber_wanted_full.csv'")


Found 0 cards.

✅ Done! Saved 0 entries to 'cyber_wanted_full.csv'


In [28]:
!pip install selenium

In [30]:
!pip install webdriver

ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver


In [32]:
# ✅ Imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time


In [36]:
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/122.0.0.0 Safari/537.36")
# Do NOT enable headless for now

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [37]:
# ✅ Load FBI Cyber Wanted Page
url = "https://www.fbi.gov/wanted/cyber"
driver.get(url)

# Wait until cards are present
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CLASS_NAME, "portal-type-person"))
)

# Parse the full rendered page
soup = BeautifulSoup(driver.page_source, 'html.parser')
cards = soup.find_all('li', class_='portal-type-person')
print(f"✅ Found {len(cards)} wanted persons")


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.104)
Stacktrace:
	GetHandleVerifier [0x0x333783+63299]
	GetHandleVerifier [0x0x3337c4+63364]
	(No symbol) [0x0x161113]
	(No symbol) [0x0x1508a0]
	(No symbol) [0x0x16e8c2]
	(No symbol) [0x0x1d519c]
	(No symbol) [0x0x1ef359]
	(No symbol) [0x0x1ce376]
	(No symbol) [0x0x19d6e0]
	(No symbol) [0x0x19e544]
	GetHandleVerifier [0x0x58e073+2531379]
	GetHandleVerifier [0x0x589372+2511666]
	GetHandleVerifier [0x0x359efa+220858]
	GetHandleVerifier [0x0x34a548+156936]
	GetHandleVerifier [0x0x350c7d+183357]
	GetHandleVerifier [0x0x33b6e8+95912]
	GetHandleVerifier [0x0x33b890+96336]
	GetHandleVerifier [0x0x32666a+9770]
	BaseThreadInitThunk [0x0x76de5d49+25]
	RtlInitializeExceptionChain [0x0x7753d09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x7753d021+561]
	(No symbol) [0x0]


In [38]:
html = driver.page_source
print(html[:1000])  # See what Selenium sees


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0x333783+63299]
	GetHandleVerifier [0x0x3337c4+63364]
	(No symbol) [0x0x160f70]
	(No symbol) [0x0x19c9a8]
	(No symbol) [0x0x1ce436]
	(No symbol) [0x0x1c9f05]
	(No symbol) [0x0x1c9486]
	(No symbol) [0x0x133a05]
	(No symbol) [0x0x133f5e]
	(No symbol) [0x0x1343ed]
	GetHandleVerifier [0x0x58e073+2531379]
	GetHandleVerifier [0x0x589372+2511666]
	GetHandleVerifier [0x0x359efa+220858]
	GetHandleVerifier [0x0x34a548+156936]
	GetHandleVerifier [0x0x350c7d+183357]
	(No symbol) [0x0x1336d0]
	(No symbol) [0x0x132edd]
	GetHandleVerifier [0x0x6c7f1c+3817180]
	BaseThreadInitThunk [0x0x76de5d49+25]
	RtlInitializeExceptionChain [0x0x7753d09b+107]
	RtlGetAppContainerNamedObjectPath [0x0x7753d021+561]
	(No symbol) [0x0]
